# imports

In [64]:
import os
import pandas as pd
from sklearn.impute import KNNImputer
import math
import numpy as np
from graphviz import Digraph
import random

## Project_path

In [65]:
project_path = '/content/drive/My Drive/Colab Notebooks/CarClassification'
print("Project directory:", project_path)

Project directory: /content/drive/My Drive/Colab Notebooks/CarClassification


# Load Datasets

In [66]:
dataset = pd.read_csv(f'{project_path}/datasets/original.train_ds.csv')
test = pd.read_csv(f'{project_path}/datasets/original.test_ds.csv')

# for col in dataset.select_dtypes(include='float'):
#     if (dataset[col] % 1 == 0).all():  # Check if all values are integers
#         print('here')
#         dataset[col] = dataset[col].astype(int)

# print(dataset)
# for col in dataset.select_dtypes(include='float'):
#     non_zero_decimal = dataset[col] % 1 != 0
#     if non_zero_decimal.any():
#         print(f"Column '{col}' has non-integer values:")
#         print(dataset.loc[non_zero_decimal, col])

# print(dataset.dtypes)

# Pre Process

## Drop Nulls

In [67]:
def drop_null(dataset) -> pd.DataFrame:
  cleaned_dataset = dataset.dropna()
  return cleaned_dataset

## Fill Numeric Values

In [68]:
def fill_numeric(dataset, method='mean', custom_val=0) -> pd.DataFrame:
  # Impute missing values in numerical columns with the mean
  numeric_dataset = dataset.select_dtypes(include=['number'])
  if(method == 'mean'):
    numeric_dataset.fillna(numeric_dataset.mean(), inplace=True)
  elif(method == 'median'):
    numeric_dataset.fillna(numeric_dataset.median(), inplace=True)
  elif(method == 'mode'):
     numeric_dataset.fillna(numeric_dataset.mode()[0], inplace=True)
  elif(method == 'custom_val'):
    numeric_dataset.fillna(value=custom_val, inplace=True)
  elif(method == 'interpolate_linear'):
    numeric_dataset.interpolate(method='linear', inplace=True)
  elif(method == 'interpolate_time'):
    numeric_dataset.interpolate(method='time', inplace=True)
  elif(method == 'KNN'):
    imputer = KNNImputer(n_neighbors=custom_val)
    data_imputed = imputer.fit_transform(numeric_dataset)
    nemeric_dataset = pd.DataFrame(data_imputed, columns=numeric_dataset.columns)
  else:
    raise Exception('wrong method')
  return numeric_dataset

## Fill Categorical

In [69]:
def fill_categorical(dataset) -> pd.DataFrame:
  categorical_dataset = dataset.select_dtypes(include=['object'])
  for col in categorical_dataset.columns:
      mode_value = categorical_dataset[col].mode()[0]
      categorical_dataset[col].fillna(mode_value, inplace=True)
  return categorical_dataset

## FFILL & BFILL

In [70]:
def forward_fulfill(dataset) -> pd.DataFrame:
  return dataset.fillna(method='ffill')

def backward_fulfill(dataset) -> pd.DataFrame:
  return dataset.fillna(method='bfill')

In [293]:
# cleaned_dataset =  pd.concat([fill_categorical(dataset), fill_numeric(dataset, method='interpolate_linear')], axis=1)
cleaned_dataset = drop_null(dataset)
# cleaned_dataset = forward_fulfill(dataset)
for col in cleaned_dataset.select_dtypes(include='float'):
    non_zero_decimal = cleaned_dataset[col] % 1 != 0
    if non_zero_decimal.any():
        print(f"Column '{col}' has non-integer values:")
        print(cleaned_dataset.loc[non_zero_decimal, col])

for col in cleaned_dataset.select_dtypes(include='float'):
    if (cleaned_dataset[col] % 1 == 0).all():  # Check if all values are integers
        print('here')
        cleaned_dataset[col] = cleaned_dataset[col].astype(int)
print(cleaned_dataset.dtypes)
# cleaned_test = pd.concat([fill_categorical(test), fill_numeric(test)], axis=1)
cleaned_test = test


# cleaned_test = drop_null(test)

Column 'mpg' has non-integer values:
1       64.2
7       58.9
11      42.2
12      51.4
13      47.1
        ... 
2379    48.7
2383    51.4
2386    44.8
2393    64.2
2396    64.2
Name: mpg, Length: 937, dtype: float64
Column 'engineSize' has non-integer values:
7       1.5
11      1.6
13      1.4
16      1.7
17      1.4
       ... 
2366    1.3
2372    1.5
2379    1.2
2383    1.4
2386    1.5
Name: engineSize, Length: 434, dtype: float64
here
here
here
here
model            object
year              int64
price             int64
transmission     object
mileage           int64
fuelType         object
tax               int64
mpg             float64
engineSize      float64
Manufacturer     object
dtype: object


<ipython-input-293-dea8f727606d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_dataset[col] = cleaned_dataset[col].astype(int)
<ipython-input-293-dea8f727606d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_dataset[col] = cleaned_dataset[col].astype(int)
<ipython-input-293-dea8f727606d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [294]:
print(cleaned_dataset)

          model  year  price transmission  mileage fuelType  tax   mpg  \
0          Aygo  2016   7500       Manual    14612   Petrol    0  69.0   
1            Up  2018   8712       Manual     1607   Petrol  145  64.2   
7         Yaris  2017   9991       Manual     2781   Petrol  150  58.9   
11      Mokka X  2017  13295       Manual     9326   Petrol  145  42.2   
12           X1  2015   8995       Manual    90000   Diesel  160  51.4   
...         ...   ...    ...          ...      ...      ...  ...   ...   
2379       Polo  2009   2495       Manual    41637   Petrol  145  48.7   
2383         Q2  2017  27999    Semi-Auto    24450   Petrol  145  51.4   
2386   EcoSport  2016  10484    Semi-Auto    19961   Petrol  160  44.8   
2393         Up  2016   6990       Manual    10876   Petrol   20  64.2   
2396   3 Series  2017  16000       Manual    34098   Diesel  145  64.2   

      engineSize Manufacturer  
0            1.0       toyota  
1            1.0   volkswagen  
7            1.

# Helper functions

In [295]:
def is_discrete(labels:pd.DataFrame.columns):
  if(labels.dtype == 'int64'):
    num_unique = labels.nunique()
    if(num_unique < 20):
      return True
  return False

def random_outcome(labels):
  model_counts = labels.value_counts()
  total_count = model_counts.sum()
  probabilities = model_counts / total_count
  outcomes = list(zip(probabilities.index, probabilities))
  random_number = random.random()

  cumulative_probability = 0
  for outcome, probability in outcomes:
    cumulative_probability += probability
    if random_number < cumulative_probability:
        return outcome
        break

def find_optimal_threshold(data, numeric_column):
  unique_values = sorted(data[numeric_column].unique())
  optimal_threshold = None
  max_info_gain = float('-inf')
  for i in range(len(unique_values) - 1):
      threshold = (unique_values[i] + unique_values[i+1]) / 2  # Calculate threshold as midpoint
      left_split = data[data[numeric_column] <= threshold]
      right_split = data[data[numeric_column] > threshold]

      info_gain = cal_information_gain(left_split[numeric_column], right_split[numeric_column])

      if info_gain > max_info_gain:
          optimal_threshold = threshold
          max_info_gain = info_gain

  if optimal_threshold is None:
    return np.float64(data[numeric_column].mean())
  return optimal_threshold

def cal_information_gain(left_labels, right_labels):

    parent_entropy = entropy(np.concatenate((left_labels, right_labels)))
    total_samples = len(left_labels) + len(right_labels)
    weighted_entropy = (len(left_labels) / total_samples) * entropy(left_labels) \
                        + (len(right_labels) / total_samples) * entropy(right_labels)

    information_gain = parent_entropy - weighted_entropy
    return information_gain
def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def calculate_information_gain(feature, parent_label, child_labels):

  child_labels = [label.root.data[feature] for label in child_labels]
  parent_entropy = entropy(parent_label[feature])
  total_samples = len(parent_label[feature])
  if total_samples == 0:
      return 0  # Avoid division by zero
  weighted_entropy = 0
  for labels in child_labels:
    branch_samples = len(labels)
    weighted_entropy += (branch_samples / total_samples) * entropy(labels)
  information_gain = parent_entropy - weighted_entropy
  return information_gain

In [296]:
# Example usage:
# Assuming 'data' is your pandas DataFrame and you want to find the optimal threshold for the 'age' column
optimal_threshold = find_optimal_threshold(cleaned_dataset, 'tax')
print('mean', cleaned_dataset['tax'].mean())
print("Optimal Threshold:", optimal_threshold)

mean 116.76409185803757
Optimal Threshold: 142.5


#Node

In [297]:
class Node:
  def __init__(self, data:pd.DataFrame, features: pd.DataFrame.columns, label, discerete_features={}, value = None) -> None:
     self.data = data
     self.features = features
     self.label = label
     self.value = value
     self.discerete_features = discerete_features

# Decision Tree

In [298]:
discrete_features = {feature: False for feature in cleaned_dataset.columns}
# discrete_features['year'] = True
# discrete_features = {feature: is_discrete(cleaned_dataset[feature]) for feature in cleaned_dataset.columns}
print(discrete_features)

{'model': False, 'year': False, 'price': False, 'transmission': False, 'mileage': False, 'fuelType': False, 'tax': False, 'mpg': False, 'engineSize': False, 'Manufacturer': False}


In [301]:
class Tree:
  def __init__(self, root:Node, depth=0, split_method='entropy') -> None:
     self.root = root
     self.depth = depth
     self.split_method = split_method
     self.branches = {}
     self.features_ig = {key: 0 for key in self.root.features}

  def build(self) -> 'Tree':

    if len(self.root.features) == 1 and len(self.root.data) > 1:
      self.root.value = random_outcome(self.root.data['model'])
      # self.root.value = self.root.data['model'].value_counts().idxmax()
    elif len(self.root.data) == 1:
      self.root.value = self.root.data['model'].value_counts().idxmax()
    else:
      selected_feature = self.feature_selector(self.split_method, self.root.features)
      self.branches = self.split(self.root.data, selected_feature)
      for branch in self.branches:
        self.branches[branch].build()

  def feature_selector(self, mode, features):
    if(mode == 'entropy'):
      features_entropy = {key: 0 for key in self.root.features}
      for feature in features:
        if(feature == 'model'):
          feature_entropy = float('+inf')
        else:
          feature_entropy = entropy(self.root.data[feature].to_list())
        features_entropy[feature] = feature_entropy

      selected_feature = min(features_entropy, key=features_entropy.get)
    elif(mode == 'information_gain'):
      features_ig = {key: 0 for key in self.root.features}
      for feature in features:
        if(feature == 'model'):
          feature_ig = float('-inf')
        else:
          branches = self.split(self.root.data, feature)
          parent_branch = self.root.data

          feature_ig = calculate_information_gain('model', parent_branch, list(branches.values()))
        features_ig[feature] = feature_ig

      selected_feature = max(features_ig, key=features_ig.get)

    return selected_feature

  def split(self, data, feature) -> tuple:
    branches = {}
    if(data[feature].dtype == 'object'):
      branches = self.split_categorical(data, feature)
    elif(self.root.discerete_features[feature]):
      branches = self.split_categorical(data, feature)
    else:
      branches = self.split_numeric(data, feature)
    return branches


  def split_numeric(self, data, feature):
    threshold = find_optimal_threshold(self.root.data, feature)
    # threshold = data[feature].mean()
    branches = {}
    left_data = data[data[feature] <= threshold].drop(feature, axis=1)
    left_node = Node(left_data, left_data.columns, label=f'{feature}.{threshold}.left', discerete_features=self.root.discerete_features)
    left_tree = Tree(left_node, depth=self.depth+1, split_method=self.split_method)
    branches[f'{feature}*{threshold}*left'] = left_tree

    right_data = data[data[feature] > threshold].drop(feature, axis=1)
    right_node = Node(right_data, right_data.columns, label=f'{feature}.{threshold}.right', discerete_features=self.root.discerete_features)
    right_tree = Tree(right_node, depth=self.depth+1, split_method=self.split_method)
    branches[f'{feature}*{threshold}*right'] = right_tree
    return branches

  def split_categorical(self, data, feature):
    branches = {}
    for data_point in set(data[feature]):
      new_data = data[data[feature] == data_point].drop(feature, axis=1)
      new_root = Node(new_data, new_data.columns, label=data_point, discerete_features=self.root.discerete_features)
      branches[f'{feature}*{data_point}'] = Tree(new_root, depth=self.depth+1, split_method=self.split_method)
    return branches


  def visualize(self):
    dot = Digraph()
    # print(self.root.features)
    self._visualize_helper(dot, self.root.label)
    dot.render(f'{project_path}/results/custom_tree', format='png', cleanup=True)

  def _visualize_helper(self, dot, parent_label):
    print(self.root.value)
    if self.root.value is not None :
      possible_label = self.root.value
    else:
      possible_label = self.root.features
    dot.node(parent_label, label=str(self.root.label))
    for branch in self.branches:
      self.branches[branch]._visualize_helper(dot, branch)
      dot.edge(parent_label, branch)


  def predict(self, data_point: pd.DataFrame):
    if self.root.value is not None:
      return self.root.value
    else:
      for branch in self.branches:
        if branch.endswith('left'):
          branch_parts = branch.split('*')
          feature = branch_parts[0]
          threshold = float(branch_parts[1])
          if(data_point.iloc[0][feature] <= threshold):
            # print(data_point)
            # print(branch)
            new_data_point = data_point.drop(feature, axis=1)
            return self.branches[branch].predict(new_data_point)

        elif branch.endswith('right'):
          branch_parts = branch.split('*')
          feature = branch_parts[0]
          threshold = float(branch_parts[1])
          if(data_point.iloc[0][feature] > threshold):
            # print(data_point)
            # print(branch)
            new_data_point = data_point.drop(feature, axis=1)
            return self.branches[branch].predict(new_data_point)
        else:
          branch_parts = branch.split('*')
          feature = branch_parts[0]
          feature_value = branch_parts[1]
          if str(data_point.iloc[0][feature]) == str(feature_value):
            # print(data_point)
            # print(branch)
            new_data_point = data_point.drop(feature, axis=1)
            return self.branches[branch].predict(new_data_point)


In [302]:

node = Node(cleaned_dataset ,cleaned_dataset.columns, 'root', discrete_features)
tree = Tree(node, split_method='entropy')
# tree.calculate_ig()
tree.build()



In [303]:
test_pd = cleaned_test.iloc[[1]].copy()
# test_pd = cleaned_dataset.iloc[[2]].copy()
print(test_pd)
result = tree.predict(test_pd)
print(result, test_pd.iloc[0]['model'])

       model  year  price transmission  mileage fuelType  tax   mpg  \
1   2 Series  2019  24590    Semi-Auto     3300   Diesel  145  48.7   

   engineSize Manufacturer  
1         2.0          BMW  
 2 Series  2 Series


In [304]:
count = 0
for i in range(len(cleaned_dataset)):
  test_pd = cleaned_dataset.iloc[[i]].copy()
  # test_pd = cleaned_dataset.iloc[[2]].copy()
  # print(test_pd)
  result = tree.predict(test_pd)
  # print(result, test_pd.iloc[0]['model'])
  if result == test_pd.iloc[0]['model']:
    count+=1
print(count/ len(cleaned_dataset))
# print(count)

0.9331941544885177


In [305]:
count = 0
for i in range(len(cleaned_test)):
  test_pd = cleaned_test.iloc[[i]].copy()
  # test_pd = cleaned_dataset.iloc[[2]].copy()
  # print(test_pd)
  result = tree.predict(test_pd)
  # print(result, test_pd.iloc[0]['model'])
  if result == test_pd.iloc[0]['model']:
    count+=1
print(count/ len(cleaned_test))

0.56


In [ ]:
tree.visualize()

In [225]:
import numpy as np
import pandas as pd

def entropy(labels):
    # Calculate the proportion of each class in the labels
    value_counts = labels.value_counts() / len(labels)
    # Filter out zero probabilities to avoid log(0) errors
    value_counts = value_counts[value_counts > 0]
    # Calculate entropy
    entropy = -np.sum(value_counts * np.log2(value_counts))
    return entropy

def information_gain(parent_label, splits):
    # Calculate the entropy of the parent node
    parent_entropy = entropy(parent_label)
    total_samples = len(parent_label)

    # Calculate the weighted entropy of the child nodes
    weighted_entropy = sum(len(split) / total_samples * entropy(split) for split in splits)

    # Calculate information gain
    information_gain = parent_entropy - weighted_entropy
    return information_gain

def calculate_information_gains(data, target_col):
    parent_entropy = entropy(data[target_col])
    information_gains = {}
    for feature in data.columns:
        if feature != target_col:
            splits = [data[data[feature] == value][target_col] for value in data[feature].unique()]
            information_gains[feature] = information_gain(data[target_col], splits)
    return information_gains

# Example usage:
# Assuming 'data' is your pandas DataFrame and 'target_col' is the name of your target variable column
data = cleaned_dataset
all = {}
for col in cleaned_dataset:

  target_col = col  # Replace 'target' with the name of your target variable column
  information_gains = calculate_information_gains(data, target_col)
  all[col] = information_gains
  print(f"Information Gains for {col}:", information_gains)


Information Gains for model: {'year': 0.22970167566054833, 'price': 3.7488683267113094, 'transmission': 0.3808796038774833, 'mileage': 4.475627913273247, 'fuelType': 0.3783389671521622, 'tax': 0.39322191018779407, 'mpg': 1.6035959204624919, 'engineSize': 1.0906926946592903, 'Manufacturer': 3.0073230099773287}
Information Gains for year: {'model': 0.2297016756605479, 'price': 1.849252268709745, 'transmission': 0.03352247444774825, 'mileage': 2.421540528816805, 'fuelType': 0.026190224766114323, 'tax': 0.37858330221795944, 'mpg': 0.3682334167013104, 'engineSize': 0.12538507458956882, 'Manufacturer': 0.0860719697360195}
Information Gains for price: {'model': 3.7488683267113023, 'year': 1.8492522687097468, 'transmission': 0.865944058002901, 'mileage': 9.24496021875471, 'fuelType': 0.5851695742933707, 'tax': 1.5911361790826906, 'mpg': 4.155712374791339, 'engineSize': 1.896821231051054, 'Manufacturer': 1.9704329740220592}
Information Gains for transmission: {'model': 0.3808796038774831, 'year

In [226]:
print(all)

{'model': {'year': 0.22970167566054833, 'price': 3.7488683267113094, 'transmission': 0.3808796038774833, 'mileage': 4.475627913273247, 'fuelType': 0.3783389671521622, 'tax': 0.39322191018779407, 'mpg': 1.6035959204624919, 'engineSize': 1.0906926946592903, 'Manufacturer': 3.0073230099773287}, 'year': {'model': 0.2297016756605479, 'price': 1.849252268709745, 'transmission': 0.03352247444774825, 'mileage': 2.421540528816805, 'fuelType': 0.026190224766114323, 'tax': 0.37858330221795944, 'mpg': 0.3682334167013104, 'engineSize': 0.12538507458956882, 'Manufacturer': 0.0860719697360195}, 'price': {'model': 3.7488683267113023, 'year': 1.8492522687097468, 'transmission': 0.865944058002901, 'mileage': 9.24496021875471, 'fuelType': 0.5851695742933707, 'tax': 1.5911361790826906, 'mpg': 4.155712374791339, 'engineSize': 1.896821231051054, 'Manufacturer': 1.9704329740220592}, 'transmission': {'model': 0.3808796038774831, 'year': 0.03352247444774781, 'price': 0.8659440580029008, 'mileage': 1.1117955592

In [227]:
average_information_gains = {}
for feature, gains in all.items():
    average_gain = sum(gains.values()) / len(gains)
    average_information_gains[feature] = average_gain

# Find the feature with the highest average information gain
highest_average_gain_feature = max(average_information_gains, key=average_information_gains.get)
highest_average_gain = average_information_gains[highest_average_gain_feature]

print("Feature with the highest average information gain:", highest_average_gain_feature)
print("Highest average information gain:", highest_average_gain)

Feature with the highest average information gain: mileage
Highest average information gain: 3.3572017181747875
